# Cosyne 2019 NWB:N Tutorial - Extracellular Electrophysiology

## Set up NWB file
NWB files require a session start time to be entered with a timezone field.

In [ ]:
from pynwb import NWBFile
from datetime import datetime
from dateutil import tz

start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz('US/Pacific'))

nwbfile = NWBFile(identifier='Mouse5_Day3',
                 session_description='mouse in open exploration and theta maze',  # required
                 session_start_time=start_time,  # required
                 experimenter='My Name',  # optional
                 session_id='session_id',  # optional
                 institution='University of My Institution',  # optional
                 lab='My Lab Name',  # optional
                 related_publications='DOI:10.1016/j.neuron.2016.12.011')  # optional

## Subject info

In [ ]:
from pynwb.file import Subject

nwbfile.subject = Subject(age='9 months', description='mouse 5',
                          species='Mus musculus', sex='M')

## Position
The `Position` object is a `MultiContainerInterface` that holds one or more `SpatialSeries` objects, which are a subclass of `TimeSeries`. Here, we put a `SpatialSeries` object called `'position'` in a `Position` object, and put that in a `ProcessingModule` named `'behavior'`.
<img src="images/position.png" width="800">

In [ ]:
import numpy as np
from pynwb.behavior import SpatialSeries, Position

position_data = np.array([
    np.linspace(0, 10, 100),
    np.linspace(1, 8, 100)]).T
spatial_series_object = SpatialSeries(
   name='position', data=position_data,
   reference_frame='unknown',
   conversion=1.0, resolution=np.nan,
   timestamps=np.linspace(0, 100) / 200)

In [ ]:
pos_obj = Position(spatial_series=spatial_series_object)
behavior_module = nwbfile.create_processing_module(
    name='behavior',
    description='data relevant to behavior')

behavior_module.add_data_interface(pos_obj)


## Write to file

In [ ]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('test_ephys.nwb', 'w') as io:
    io.write(nwbfile)


## Electrodes table
Extracellular electrodes are stored in a `electrodes`, which is a `DynamicTable`. `electrodes` has several required fields: x, y, z, impedence, location, filtering, and electrode_group. Here, we also demonstate how to add optional columns to a table by adding the `'label'` column.<img src="images/electrodes_table.png" width="300">

In [ ]:
nwbfile.add_electrode_column('label', 'label of electrode')
shank_channels = [4, 3]

electrode_counter = 0
device = nwbfile.create_device('implant')
for shankn, nelecs in enumerate(shank_channels):
    electrode_group = nwbfile.create_electrode_group(
       name='shank{}'.format(shankn),
       description='electrode group for shank {}'.format(shankn),
       device=device,
       location='brain area')
    for ielec in range(nelecs):
        nwbfile.add_electrode(
           x=5.3, y=1.5, z=8.5, imp=np.nan,
           location='unknown', filtering='unknown',
           group=electrode_group,
           label='shank{}elec{}'.format(shankn, ielec))
        electrode_counter += 1

all_table_region = nwbfile.create_electrode_table_region(
  list(range(electrode_counter)), 'all electrodes')

## LFP
`LFP` is another `MultiContainerInterface`. It holds one or more `ElectricalSeries` objects, which are `TimeSeries`. Here, we put an `ElectricalSeries` named `'lfp'` in an `LFP` object, in a `ProcessingModule` named `'ecephys'`.
<img src="images/lfp.png" width="800">

In [ ]:
from pynwb.ecephys import ElectricalSeries, LFP
lfp_data = np.random.randn(100, 7)
ecephys_module = nwbfile.create_processing_module(
    name='ecephys',
    description='extracellular electrophysiology data')
ecephys_module.add_data_interface(
LFP(ElectricalSeries('lfp', lfp_data, all_table_region, 
rate=1000., resolution=.001, conversion=1.)))

## Spike Times
Spike times are stored in another `DynamicTable` of subtype `Units`. The main `Units` table is at `/units` in the HDF5 file. You can add columns to the `Units` table just like you did for `electrodes`.

In [ ]:
for shankn, channels in enumerate(shank_channels):
    for n_units_per_shank in range(np.random.poisson(lam=5)):
        n_spikes = np.random.poisson(lam=10)
        spike_times = np.abs(np.random.randn(n_spikes))
        nwbfile.add_unit(spike_times=spike_times)

## Trials
Trials is another `DynamicTable` that lives an `/intervals/trials`.

In [ ]:
nwbfile.add_trial_column('correct', description='correct trial')
nwbfile.add_trial(start_time=1.0, stop_time=5.0, correct=True)
nwbfile.add_trial(start_time=6.0, stop_time=10.0, correct=False)

## Write and read
Data arrays are read passively from the file. That means `TimeSeries.data` does not read the entire data object, but presents an h5py object that can be indexed to read data. Index this array just like a numpy array to read only a specific section of the array, or use the `[:]` operator to read the entire thing.

In [ ]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('test_ephys.nwb', 'w') as io:
    io.write(nwbfile)

with NWBHDF5IO('test_ephys.nwb', 'r') as io:
    nwbfile2 = io.read()

    print(nwbfile2.modules['ecephys']['LFP'].electrical_series['lfp'].data[:])

## Accessing data regions
You can easily read subsections of datasets

In [ ]:
io = NWBHDF5IO('test_ephys.nwb', 'r')
nwbfile2 = io.read()

print('section of lfp:')
print(nwbfile2.modules['ecephys']['LFP'].electrical_series['lfp'].data[:10,:5])
print('')
print('')
print('spike times from first unit:')
print(nwbfile2.units['spike_times'][0])
io.close()